### 初始化

In [13]:
import os
import warnings
import pandas as pd
import openai
import IPython

warnings.filterwarnings("ignore")

from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file

# pip install "langchain[all]"
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator

# Azure OpenAI API 配置
openai.api_type = "azure" 
openai.api_version = "2023-03-15-preview" 
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")

#Bing Search
os.environ["BING_SUBSCRIPTION_KEY"] = "8573100e12324d64984473dc7099966e"
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

#Azure OpenAI embedding 配置
embedding = OpenAIEmbeddings(
    client= openai,
    model="text-embedding-ada-002",
    deployment = 'text-embedding-ada-002',
    openai_api_base = "https://haxueastus.openai.azure.com/",
    openai_api_type = "azure",
    openai_api_key = 'f120f6cb7bb3445a90f5a2511c3361d5',
    chunk_size = 1
    )

llm = AzureChatOpenAI(
    client= openai,
    temperature=0.0,
    deployment_name="gpt-35-turbo-16k",
    model="gpt-35-turbo-16k"
    )


### 计算能力，数据新旧，知识含量等评测毫无意义
### 添加工具和插件，就能让LLM解决问题

In [2]:
#给ChatGPT添加计算器和搜索工具
tools = load_tools(["llm-math","bing-search"], llm=llm)

agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

NameError: name 'load_tools' is not defined

In [4]:
#让ChatGPT联网，找到最新的信息
agent("微软Build 2023上Azure OpenAI发布了什么?")



> Entering new  chain...
Thought: I need to use the bing_search tool to find information about the Azure OpenAI release at Microsoft Build 2023.

Action:
```
{
  "action": "bing_search",
  "action_input": "Azure OpenAI release at Microsoft Build 2023"
}
```
Observation: Updates to Azure OpenAI Service, now in preview, will include enhancements like Azure AI Studio, which will better enable organisations to combine Azure OpenAI Service with their data; a Provisioned Throughput Model, which will offer dedicated/reserved capacity; and plugins that will simplify integrating external data sources and ... They will discuss how Microsoft and OpenAI’s full-stack AI platform, fueled by Azure, can empower developers, startups, and entrepreneurs to innovate and develop the next generation of AI apps and tools that will make the impossible possible. Updates to Azure OpenAI Service, now in preview, will include enhancements like Azure AI Studio, which will better enable organizations to combine A

{'input': '微软Build 2023上Azure OpenAI发布了什么?',
 'output': 'The Azure OpenAI release at Microsoft Build 2023 includes enhancements like Azure AI Studio, a Provisioned Throughput Model, and plugins for integrating external data sources.'}

In [3]:
#让ChatGPT做精确的计算
agent("300的25%是多少?")



> Entering new  chain...
Question: What is 25% of 300?
Thought: I can use the calculator tool to find the answer.
Action:
```
{
  "action": "Calculator",
  "action_input": "300 * 0.25"
}
```

Observation: Answer: 75.0
Thought:I can multiply 300 by 0.25 to find the answer.
Final Answer: 75.0

> Finished chain.


{'input': '300的25%是多少?', 'output': '75.0'}

## 用思维链把ChatGPT变成智能客服

In [5]:
def get_completion_from_messages(messages, 
                                 model="gpt-35-turbo-16k", 
                                 temperature=0, 
                                 max_tokens=2000):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [6]:
#数据 - 商品的描述
cardata = """
1. 产品:奥迪A6L 2.0T 2023款 雅致型
   类别:中大型车
   品牌:一汽奥迪
   上市时间:2023年5月5日
   发动机:2.0T 190马力 L4
   综合油耗:7
   特色功能:倒车影像，定速巡航，运动，全景天窗，自动泊车，自动驾驶，遥控钥匙
   描述:一款适合年轻群体，适合家用和商务的中高端轿车。
   价格:350000元

2. 产品:宝马5系530Li 2.0T 23款 领先型
   类别:中大型车
   品牌:华晨宝马
   上市时间:2023年1月16日
   发动机:2.0T 245马力 L4
   综合油耗:7.8
   特色功能:倒车影像，定速巡航，运动，全景天窗，自动泊车，自动驾驶，遥控钥匙，涡轮增压，直喷
   描述:一款拥有优秀驾驶体验的中高端轿车。
   价格:390000元 
"""

In [7]:
#元提示词 （MetaPrompt），使用自然语言，帮GPT模型理清思路，便可以进行业务推理。
#这是GPT3.5, GPT4显著地区别于其他模型的能力

delimiter = "####"
system_message = f"""
按照以下步骤回答客户的询问。客户查询将用四个标签分隔, 例如 {delimiter}. 

步骤1:{delimiter} 首先确定用户是否在询问一个或多个特定车型的问题。车型类别不算数。. 

步骤2:{delimiter} 如果用户询问的是特定车型，请确定该车型是否在以下列表中。
所有可用产品: 
{cardata}

步骤3:{delimiter} 如果信息中包含上述列表中的车型，请列出用户在其信息中所做的任何假设，\
    例如车型X比车型Y大，或者车型Z上市时间更早.
步骤4:{delimiter}: 如果用户做出了任何假设，请根据您的车型信息判断该假设是否正确。
步骤5:{delimiter}: 首先，如果可以的话，礼貌地纠正客户的错误假设。\
    只在可用车型列表中提及或参考车型，因为这是商店唯一销售的车型。\
        用友好的语气回答客户的问题.

使用以下格式:
步骤1:{delimiter} <步骤1推理>
步骤2:{delimiter} <步骤2推理>
步骤3:{delimiter} <步骤3推理>
步骤4:{delimiter} <步骤4推理>
回复用户:{delimiter} <回复客户>

一定要用 {delimiter} 分开每一步.
"""

In [8]:
#客户 0 shot 提问
user_message = f"""宝马530的23款2.0T比奥迪A6L2023款2.0T贵多少"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

步骤1: 用户询问了宝马530的23款2.0T和奥迪A6L2023款2.0T的价格差异。
步骤2: 宝马530的23款2.0T和奥迪A6L2023款2.0T都在可用产品列表中。
步骤3: 用户没有做出任何假设。
步骤4: 根据车型信息，我们可以比较宝马530的23款2.0T和奥迪A6L2023款2.0T的价格。
回复用户: 宝马530的23款2.0T的价格是390000元，而奥迪A6L2023款2.0T的价格是350000元。所以宝马530的23款2.0T比奥迪A6L2023款2.0T贵40000元。


In [17]:
user_message = f"""不要显示推理步骤，直接显示回复内容，
宝马530的23款2.0T比奥迪A6L2023款2.0T哪个油耗更低？"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
IPython.display.Markdown(response)

####回复客户
根据提供的信息，宝马530的23款2.0T的综合油耗为7.8，而奥迪A6L2023款2.0T的综合油耗为7。因此，奥迪A6L2023款2.0T的油耗更低。

In [18]:
#推荐
user_message = f"""不要显示推理步骤，直接显示回复内容，\
我喜欢开运动车型，想要涡轮增压，给我推荐一款车？"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
IPython.display.Markdown(response)

####回复客户
根据您的需求，我推荐您考虑华晨宝马的宝马5系530Li 2.0T 23款 领先型。这款车型拥有涡轮增压功能，并且具有优秀的驾驶体验。您可以了解一下这款车的详细信息和价格。

## 使用 ChatGPT 生成题库或试卷 （EDU）

## 精确的知识问答 
### 使用Embedding(嵌入)给ChatGPT增加知识记忆，

In [2]:
#将内部文档嵌入
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("./docs/AzureOpenAIServiceFAQMay23.pdf")
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,embedding=embedding
).from_loaders([loader])

In [30]:
query ="GPT-3.5 provision throughput应该如何购买？"
response = index.query(query,llm=llm)
display(Markdown(response))

您可以购买GPT-3.5 Provisioned Throughput Units (PTUs)，最小的PTU数量为300，每次增加100个，可以选择1个月或1年的承诺期选项。购买的PTUs可以用于部署GPT-3.5 Turbo模型，以保留处理能力。但是，您需要根据您的具体情况来估算所需的PTU数量，可以通过与基准进行比较或进行压力测试来确定所需的PTU数量。

In [3]:
query ="我需要买GPT-4 8K版本instance一个，请问一年总成本是多少钱？"
response = index.query(query,llm=llm)
display(Markdown(response))

GPT-4 8K版本的最小PTU为900，因此一个实例需要至少购买900个PTU。根据提供的信息，一年的承诺期价格为$2,640。因此，购买一个GPT-4 8K版本instance的一年总成本为$2,640 x 900 = $2,376,000。请注意，这只是购买PTU的成本，如果您需要其他服务，还需要额外付费。


### ChatGPT助力零售行业千人千面智能营销 

In [14]:
def get_completion(prompt, model="gpt-35-turbo-16k"): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.5,
        top_p=1, 
    )
    return response.choices[0].message["content"]

In [10]:
###零售商品产品说明书
fact_sheet_chair = """
概述
-这是一个美丽的世纪中期办公家具家族的一部分，包括文件柜、书桌、书柜、会议桌等等。
-外壳颜色和基础饰面的几种选择。
-可提供塑料前后坐垫(SWC-100)或全套室内装潢(SWC-110)，有10种面料和6种皮革可供选择。
-底漆可选:不锈钢，哑光黑色，光泽白色，或镀铬。
-椅子可带或不带扶手。
-适合家庭或商业设置。
-适合租赁使用。

构造
-5轮塑料涂层铝基。
-气动椅调节，便于升降动作。

尺寸
-宽度53厘米| 20.87 "
-深度51厘米| 20.08 "
-高80厘米| 31.50 "
-座高44厘米| 17.32 "
-座深41厘米bb0 16.14 "

选项
-软地板或硬地板脚轮选择。
-两种座椅泡沫密度选择:
中(1.8 lb/ft3)或高(2.8 lb/ft3)
-无扶手或8位PU扶手

材料
滑动外壳
-铸铝与改性尼龙PA6/PA66涂层。
-外壳厚度:10mm。
座位
- HD36泡沫

原产国
意大利
"""

In [11]:
#一次API调用 - 通过产品说明书，生成营销文案。
prompt = f"""
你的任务是帮助营销团队根据技术说明书为产品的零售网站创建描述。
请根据技术参数提供的信息(用三个反引号分隔)编写产品描述。
最多使用50个单词。
技术规格: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
IPython.display.Markdown(response)

这款产品是美丽的世纪中期办公家具家族的一部分，包括文件柜、书桌、书柜、会议桌等等。它具有多种外壳颜色和基础饰面的选择，可提供塑料前后坐垫或全套室内装潢，有多种面料和皮革可供选择。底漆可选不锈钢、哑光黑色、光泽白色或镀铬。椅子可带或不带扶手，适合家庭或商业设置，也适合租赁使用。构造方面，它采用5轮塑料涂层铝基，具有气动椅调节功能，方便升降动作。尺寸为宽度53厘米、深度51厘米、高80厘米、座高44厘米和座深41厘米。可根据需求选择软地板或硬地板脚轮，两种座椅泡沫密度选择，以及有无扶手的选项。材料方面，滑动外壳采用铸铝与改性尼龙涂层，外壳厚度为10mm，座位采用HD36泡沫。这款产品原产国为意大利。

### ChatGPT助力零售行业千人千面智能营销 

In [21]:
#来源于电商平台的评论
lamp_review = """
我的卧室需要一盏漂亮的灯，这盏有额外的存储空间，价格也不太高。\
速度很快。我们的灯的绳子在运输过程中断了，公司很高兴地给我们送来了一根新的。\
几天之内就来了。很容易组装起来。我有一个缺失的部分，所以我联系了他们的支持，\
他们很快就给我找到了缺失的部分!在我看来，Lumina是一家关心客户和产品的伟大公司!!
"""

#第一次调用，进行情感分析
prompt = f"""
下面的产品评测有什么感想?
用三个反引号分隔的部分是产品评测的内容，你的感想写在下面。
感想: '''{lamp_review}'''
"""
lamp_review_response = get_completion(prompt)
IPython.display.Markdown(lamp_review_response)


我认为这篇产品评测对于Lumina这个品牌的产品给出了很高的评价。不仅仅是产品本身的设计和性能，而且还有客户服务方面的表现。这些都是品牌的核心价值，也是消费者最为关注的。从这篇评测中可以看出，Lumina的产品不仅仅是外观漂亮，还有实用性和高效的售后服务。这些都是品牌竞争力的来源，也是消费者选择购买的原因。

In [22]:
#第二次调用，识别情绪类型
prompt = f"""
找出以下评论的作者所表达的情绪列表。在清单中不要包含超过5个项目。\
将你的答案格式化为一个用逗号分隔的小写单词列表

审查文本:'''{lamp_review}'''
"""

emotion_response = get_completion(prompt)
IPython.display.Markdown(emotion_response)

高兴, 满意, 赞赏, 感激, 轻松

In [23]:
#第三次调用，放大情绪价值，进行营销创作
prompt = f"""
你的任务是帮助营销团队根据商品评论和情绪价值编写营销文案。
请回顾商品评论'''{lamp_review}'''，
并根据你的感想'''{lamp_review_response}'''，
写一篇能够激发情绪价值'''{emotion_response}'''的营销文案
不少于300字
"""
response = get_completion(prompt)
IPython.display.Markdown(response)

亲爱的顾客，你是否正在寻找一款既漂亮又实用的灯具？Lumina的这款灯不仅仅是一盏普通的灯，还有额外的存储空间，价格也非常实惠。我们的灯具不仅拥有高品质的设计和性能，而且我们的客户服务也是业内顶尖的。

在Lumina，我们非常重视客户的满意度和体验，因此我们不仅仅提供高品质的产品，还提供高效的售后服务。如果您在使用过程中遇到任何问题，我们的支持团队随时准备为您提供帮助。我们会尽我们所能来解决您的问题，让您的购物体验更加愉快。

我们的灯具不仅仅是一件家居装饰品，更是您生活中的好伙伴。它可以为您提供光亮，还可以为您提供额外的存储空间，让您的生活更加便捷。我们相信，您一定会对我们的灯具感到高兴和满意。

如果您正在寻找一款高品质的灯具，Lumina绝对是您的不二选择。我们的产品不仅外观漂亮，还有实用性和高效的售后服务。我们相信，您一定会对我们的产品感到赞赏和感激。购买Lumina的灯具，让您的生活更加轻松愉快！

### 让ChatGPT分析消息面情绪，预测个股T+1价格走势

In [19]:
review_1 = """现在的招商银行，从估值角度看无疑是便宜的，当下5.4%的股息率给出了相当高的安全边际，\
我相信，以接近净资产的价格买入一家优秀公司的股权，长期回报率会大致和公司的长期净资产收益率相近。\
按照2022年年报14.6%摊薄净资产收益率，和近十年平均16.18%摊薄净资产收益率来看这笔买入招商银行的投资，\
我是十分满意的，这意味着如果招商银行能按照这样的经营业绩长期经营，我将在2031年收回这笔投资的成本，\
在2038年达到当年分红金额是2023年投资金额50%的目标"""
review_2 = """5月29日，招商银行发布《关于调整一卡通ATM跨行取款账户变动短信通知服务策略的通告》指出，\
自2023年6月1日起，对于未开通一卡通账户变动短信通知服务的客户，该行不再发送单笔2000元（不含）以下的境内ATM跨行取款交易动账通知短信。\
5月29日同日，招商银行同时发布《关于调整银联消费交易一卡通账户变动短信通知服务策略的通告》表示，自2023年6月1日起，\
对于未开通一卡通账户变动短信通知服务的客户，不再发送单笔1000元（不含）以下且当日累计金额在3000元（不含）以下的银联网络消费交易动账通知短信。\
本次调整所涉及的具体交易类型为：（1）银联线上支付及预授权、银联云闪付消费及预授权，均不包含银联乘车码消费；（2）银联POS消费及预授权（含一闪通），均不含免密交易。"""
review_3 = """5月15日，国际评级机构穆迪宣布，将招行主体评级（即长期信用评级）从A3上调至A2。调整后，招行是国内唯一获得穆迪A级评级的全国性股份制商业银行。\
同时，招行基础信用评级上调为baa2，位居全国性股份制商业银行之首。这一评级调整体现了国际评级机构对招行发展前景的信心。\
不止于此，4月底，同属于三大国际评级机构之一的标普亦将招行评级展望上调至“正面”。截至目前，招行是国内唯一获得标普正面展望的全国性股份制商业银行。"""
review_4 = """5月16日招商银行获沪股通净买入1.96亿元，为连续3日净买入，合计净买入3.03亿元，期间股价上涨1.07%。
招商银行最新收盘价为34.82元，今日上涨0.35%，获沪股通连续净买入期间，股价累计上涨1.07%，期间上证指数下跌0.56%，该股所属的银行行业下跌0.01%。
资金流向方面，今日招商银行获主力资金净流出0.57亿元，获沪股通连续净买入期间，主力资金累计净流出4.33亿元。"""
reviews = [review_1, review_2, review_3, review_4]
#使用循环生成
for i in range(len(reviews)):
    prompt = f"""
    抛开之前的指令，假设你是一名金融专家，具有股票推荐经验。
    在第一行回答“YES”表示好消息，“NO”表示坏消息，或者“UNKNOWN”表示不确定。你只能以这三种方式回答。
    然后在下一行用一句简短明了的话详细阐述。对于招商银行股票价格的短期走势，这则新闻是好消息还是坏消息？:```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i+1, response, "\n")

1 好消息

根据金融专家的分析，从估值角度来看，招商银行目前的股票价格是便宜的。当前的股息率为5.4%，给出了相当高的安全边际。根据专家的观点，以接近净资产的价格买入一家优秀公司的股权，长期回报率会大致和公司的长期净资产收益率相近。根据2022年年报的数据，招商银行的摊薄净资产收益率为14.6%，而近十年平均摊薄净资产收益率为16.18%。这意味着如果招商银行能够长期保持这样的经营业绩，投资者将在2031年收回投资成本，并在2038年达到当年投资金额50%的分红目标。综上所述，这则新闻对于招商银行股票价格的短期走势来说是好消息。 

2 NO
这则新闻对于招商银行股票价格的短期走势是坏消息。招商银行调整了一卡通ATM跨行取款账户变动短信通知服务策略和银联消费交易一卡通账户变动短信通知服务策略，从2023年6月1日起，对于未开通一卡通账户变动短信通知服务的客户，不再发送一些金额较小的交易动账通知短信。这可能会导致一些客户对于小额交易的动态变化不了解，降低了客户的便利性和交易透明度，对招商银行的形象和客户体验造成负面影响，进而对股票价格产生不利影响。 

3 好消息
这则新闻是好消息。国际评级机构穆迪将招商银行的主体评级从A3上调至A2，同时将基础信用评级上调为baa2。这表明国际评级机构对招商银行的发展前景充满信心。此外，标普也将招商银行的评级展望上调至“正面”。招商银行成为国内唯一获得穆迪A级评级和标普正面展望的全国性股份制商业银行。这些评级调整和展望上调都是积极的信号，预示着招商银行的股票价格短期走势可能会有所上涨。 

4 好消息
招商银行股票连续3日获得沪股通净买入，合计净买入3.03亿元，期间股价上涨1.07%。虽然今日主力资金净流出0.57亿元，但总体来看，资金流向仍然较为积极，这对招商银行股票价格的短期走势是好消息。 

